In [1]:
import DBHandler


In [5]:
from DBHandler import Handler
from redis import Redis
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras import models
from keras.preprocessing.sequence import TimeseriesGenerator
import keras




class Modeling():

    def __init__(self, key):
        redis = Redis()
        handler = Handler(redis)
        self.scaler = MinMaxScaler()
        self.data = handler.load_all(key)
        self.data = pd.DataFrame(self.data)
        self.data[['close']] = self.scaler.fit_transform(self.data[['close']])
        self.predictAhead = 5
        self.time_steps = 15

    def graph(self):
        priceList = self.data['close'].values
        priceList = np.append(priceList, self.pred)
        priceList = np.reshape(priceList, (-1, 1))
        priceList = self.scaler.inverse_transform(priceList)
        minList = self.data['relative_minute'].values
        for i in range(self.predictAhead):
            minList = np.append(minList, (minList[-1] + 1))
        # priceList = np.reshape(priceList, (-1,1))
        # minList = np.reshape(minList, (-1, 1))
        dfprice = priceList[:-5]
        dfmin = minList[:-5]
        predprice = priceList[-5:]
        predmin = minList[-5:]
        plt.plot(dfmin, dfprice)
        plt.plot(predmin, predprice)
        plt.show()

    def getPredictions(self, currentKey):
        path = currentKey + ".h5"
        model = models.load_model(path)

        price = self.data['close'].values

        # --------Predictions------------#
        price = price.reshape(-1)
        predictions = price[-self.time_steps:]
        for derp in range(self.predictAhead):
            x = predictions[-self.time_steps:]
            x = x.reshape((1, self.time_steps, 1))
            out = model.predict(x)[0][0]
            predictions = np.append(predictions, out)
        predictions = predictions[self.time_steps:]
        # --------------------------------#
        predictions = np.asarray(predictions)
        self.pred = np.reshape(predictions, (-1, 1))
        return(self.scaler.inverse_transform(self.pred))



In [21]:
redis = Redis()
handler = Handler(redis)
scaler = MinMaxScaler()
data = handler.load_all('USA')
data = pd.DataFrame(data)
print(data)
data[['close']] = scaler.fit_transform(data[['close']])
predictAhead = 5
time_steps = 15

        high      low     open  minute  relative_minute    close
0    1.12410  1.12386  1.12410      24                0  1.12380
1    1.12410  1.12380  1.12380      25                1  1.12410
2    1.12416  1.12390  1.12410      26                2  1.12404
3    1.12410  1.12380  1.12404      27                3  1.12405
4    1.12417  1.12390  1.12405      28                4  1.12409
..       ...      ...      ...     ...              ...      ...
271  1.12420  1.12400  1.12420      55              271  1.12415
272  1.12423  1.12400  1.12415      56              272  1.12419
273  1.12430  1.12400  1.12419      57              273  1.12400
274  1.12430  1.12400  1.12400      58              274  1.12421
275  1.12430  1.12400  1.12421      59              275  1.12412

[276 rows x 6 columns]


In [8]:
print(data)

        high      low     open  minute  relative_minute     close
0    1.12410  1.12386  1.12410      24                0  0.822511
1    1.12410  1.12380  1.12380      25                1  0.952381
2    1.12416  1.12390  1.12410      26                2  0.926407
3    1.12410  1.12380  1.12404      27                3  0.930736
4    1.12417  1.12390  1.12405      28                4  0.948052
..       ...      ...      ...     ...              ...       ...
271  1.12420  1.12400  1.12420      55              271  0.974026
272  1.12423  1.12400  1.12415      56              272  0.991342
273  1.12430  1.12400  1.12419      57              273  0.909091
274  1.12430  1.12400  1.12400      58              274  1.000000
275  1.12430  1.12400  1.12421      59              275  0.961039

[276 rows x 6 columns]


In [9]:
close = data['close'].values
trainSize = 0.8

####SPLIT#####

trainSplit = int(len(data) * trainSize)
train, test = close[:trainSplit], close[trainSplit:]
print(len(train), len(test))

220 56


In [10]:
train = np.reshape(train, (-1,1))
test = np.reshape(test, (-1,1))

In [11]:
train_series = TimeseriesGenerator(train, train, length=time_steps, batch_size=20)     
test_series = TimeseriesGenerator(test, test, length=time_steps, batch_size=1)

In [12]:
num_epochs = 25
model = Sequential()
model.add(
    LSTM(10,
        activation='relu',
        input_shape=(time_steps,1))
)
model.add()
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 25
model.fit_generator(train_series, epochs=num_epochs, verbose=1)

Epoch 1/25
11/11 [==============================] - 1s 76ms/step - loss: 0.3433
Epoch 2/25
11/11 [==============================] - 0s 10ms/step - loss: 0.2713
Epoch 3/25
11/11 [==============================] - 0s 12ms/step - loss: 0.2125
Epoch 4/25
11/11 [==============================] - 0s 11ms/step - loss: 0.1524
Epoch 5/25
11/11 [==============================] - 0s 13ms/step - loss: 0.0949
Epoch 6/25
11/11 [==============================] - 0s 11ms/step - loss: 0.0466
Epoch 7/25
11/11 [==============================] - 0s 13ms/step - loss: 0.0140
Epoch 8/25
11/11 [==============================] - 0s 17ms/step - loss: 0.0085
Epoch 9/25
11/11 [==============================] - 0s 11ms/step - loss: 0.0087
Epoch 10/25
11/11 [==============================] - 0s 11ms/step - loss: 0.0074
Epoch 11/25
11/11 [==============================] - 0s 16ms/step - loss: 0.0074
Epoch 12/25
11/11 [==============================] - 0s 13ms/step - loss: 0.0073
Epoch 13/25
11/11 [==================

In [13]:
import seaborn as sb
import matplotlib.pyplot as plt

In [14]:
pred = model.predict_generator(test_series)

In [15]:
testVals = test[time_steps:]

In [17]:
pred = scaler.inverse_transform(pred)

In [18]:
testVals = scaler.inverse_transform(testVals)

In [22]:
squared = []
for p, t in zip(pred, testVals):
    squared.append((t-p)**2)
    mean = sum(squared)/len(squared)
    
print(mean)

[2.73395545e-08]
